# Search SDSS sample for observed 12CO 1-0 in ALMA archival data
## Search the ALMA archive using the `ALMAxmatch` module. Specifically we need the `runTargetQueryWithLines` method

In [1]:
#from astropy.table import Table
# read in sample
#MANGA_sample = Table.read('/Users/thbrown/samples/SDSS/dr15/manga/spectro/redux/v2_4_3/ryan-drpall-v2_4_3_unique_gals_only.csv')

The following are bash commands included during set up since terminal in the jupyter notebook server was unresponsive. Current solution is using the command line in a separate (but parallel) desktop session.

In [2]:
#! pip install pymysql

In [3]:
#! pip install astroquery

### Generate SDSS superset

 - Must have spectroscopic redshift (z_spec < 0.5), 3" fibre aperture is ~18kpc at z = 0.5
 - Must have MPA/JHU stellar masses (log(mass) > 1)
 - Must have declination less than 47 (maximum ALMA declination)

"The adopted upper declination limit for ALMA is  + 47∘" (https://almascience.eso.org/proposing/proposers-guide) 

In [1]:
import pymysql
import numpy as np

#Query MySQL for mock survey statmorph results
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')
#x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47'
x = 'SELECT DISTINCT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u, dr15_ALMA_spaxels a WHERE u.objID = a.objID AND u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47'
#x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47 AND u.ra > 20 and u.ra < 35'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
data = np.array(db_data, dtype = float).T
table_id = np.array(db_data, dtype = str).T[0]

In [15]:
len(data[0])

51

In [3]:
ra_sdss = data[1]
dec_sdss = data[2]

In [4]:
print(ra_sdss[0],dec_sdss[0])

49.82143689 -0.96964645


In [5]:
z_sdss = data[4]

In [6]:
z_sdss[0]

0.053812

### Set path to custom `astroquery` module
This is a temporary hack to "use a version of ```astroquery``` that does not have a bug in the release date column. 

### Import the `ALMAxmatch` tool
requires adding to the python path for now

In [48]:
import sys
sys.path

['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/',
 '/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/notebooks',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/arc/home/swilkinson/.local/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/arc/home/swilkinson/.ipython']

In [7]:
import sys
#sys.path = ['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/tools/ArchiveDev'] + sys.path
sys.path = ['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/'] + sys.path
import ALMAxmatch
#from ALMAxmatch import archiveSearch
import importlib
ALMAxmatch = importlib.reload(ALMAxmatch) # reloads module instead of restarting kernel!
archiveSearch = ALMAxmatch.archiveSearch

## Run the archive search
1. Create list of lists containing `SkyCoord` objects and search radii ie. [SkyCoord(ra,dec), searchRadius]*
2. Define the line rest frequencies (names are optional, output defaults to *line 0*, *line 1*, ...)
2. Pass tuple array along with lines of interest to `runTargetQueriesWithLines`, selecting only science data

### 1. `archiveSearch` takes either a tuple ('ra dec', 'radius') or lists of tuples.

In [10]:
from astropy import units as u
from astropy.coordinates import SkyCoord

# archive search impact parameter
#searchradius = '20arcsec'
#searchradius = '3arcsec'
searchradius = '30arcsec'

# SkyCoord objects
#targetCoords = [[SkyCoord(ra=ra*u.degree, dec=dec*u.degree), searchradius]
#                   for ra, dec in zip(ra_sdss, dec_sdss)]

targetCoords = [[SkyCoord(ra=ra*u.degree, dec=dec*u.degree), 30*u.arcsec] for ra, dec in zip(ra_sdss, dec_sdss)]


Generate target list in `mangaLineQuery`

In [51]:
# search using SkyCoord array
sdssALMAQuery = archiveSearch(targets=targetCoords)

display the target dictionary

In [52]:
display(sdssALMAQuery.targets)

{'coord=(49.82143689 deg -0.96964645 deg) radius=0.008333333333333333deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (49.82143689, -0.96964645)>,
  <Angle 30. arcsec>],
 'coord=(49.78215908 deg 0.95595472 deg) radius=0.008333333333333333deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (49.78215908, 0.95595472)>,
  <Angle 30. arcsec>],
 'coord=(49.92932926 deg 0.56545578 deg) radius=0.008333333333333333deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (49.92932926, 0.56545578)>,
  <Angle 30. arcsec>],
 'coord=(357.65176944 deg -1.12805687 deg) radius=0.008333333333333333deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (357.65176944, -1.12805687)>,
  <Angle 30. arcsec>],
 'coord=(322.98051569 deg 0.21380748 deg) radius=0.008333333333333333deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (322.98051569, 0.21380748)>,
  <Angle 30. arcsec>],
 'coord=(323.40752218 deg 1.07095399 deg) radius=0.008333333333333333deg': [<SkyCoord (ICRS): (ra, dec) in deg
      (323.40752218, 1.07095399)>,
  <Angle 30

### 2. Define lines of interest

Search for the following lines:

1. $^{12}$CO ($J=1-0$) @ 115.27120180 GHz rest frequency
2. $^{12}$CO ($J=2-1$) @ 230.53800000 GHz rest frequency
3. $^{12}$CO ($J=3-2$) @ 345.79598990 GHz rest frequency

In [8]:
# rest frame frequencies
rf_12CO_10 = 115.27120180 # 12CO J=1-0
#rf_12CO_21 = 230.53800000 # 12CO J=2-1
#rf_12CO_32 = 345.79598990 # 12CO J=3-2

rest_frequencies = [rf_12CO_10] # list of rest frequencies

line_names = ['12CO J=1-0'] # column names for observed boolean flags 

Run the query, pulling only science data where lines are observed

### 3. Search for ALMA data only using `runQueries`

In [54]:


sdssALMAQuery.runQueries(science=True, public=True, radius = 30*u.arcsec)

ALMA archive querying: 100%|██████████| 51/51 [01:31<00:00,  1.80s/ target]


coord=(42.46662204 deg -0.07121246 deg) radius=0.008333333333333333deg : No result
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.
Dev alert: "sensitivity_10kms" column has units so it may not be wise to completely replace it in _parseLineSensitivities anymore.


In [55]:
display(sdssALMAQuery.queryResults)

{'coord=(49.82143689 deg -0.96964645 deg) radius=0.008333333333333333deg': <Table length=4>
      obs_publisher_did      obs_collection ... Polarizations [4]
                                            ...                  
            str33                 str4      ...        str5      
 --------------------------- -------------- ... -----------------
 ADS/JAO.ALMA#2018.1.00541.S           ALMA ...    XX YY .. XX YY
 ADS/JAO.ALMA#2018.1.00541.S           ALMA ...    XX YY .. XX YY
 ADS/JAO.ALMA#2018.1.00541.S           ALMA ...    XX YY .. XX YY
 ADS/JAO.ALMA#2018.1.00541.S           ALMA ...    XX YY .. XX YY,
 'coord=(49.78215908 deg 0.95595472 deg) radius=0.008333333333333333deg': <Table length=4>
      obs_publisher_did      obs_collection ... Polarizations [4]
                                            ...                  
            str33                 str4      ...        str5      
 --------------------------- -------------- ... -----------------
 ADS/JAO.ALMA#2018.1.005

In [56]:
n=0 #counter
for targetname in sdssALMAQuery.targets.keys():
    if len(sdssALMAQuery.queryResults[targetname])>=1:
        n = n+1
        display(sdssALMAQuery.queryResults[targetname])

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2018.1.00541.S,ALMA,JAO,ALMA,uid://A001/X133d/Xe73.source.587731511545626673_8081-3704.spw.19,image,2,587731511545626673_8081-3704,49.82144164995129,-0.9696313100004825,0.01578969678928915,Circle ICRS 49.821442 -0.969631 0.007895,--,--,128,1,2,1.6159977228975666,58478.984331,58478.993282,665.28,665.28,0.0027582121095871756,0.0028095113795630995,3414.2097741544376,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X133d/Xe73,applicati,--,2018.1.00541.S,Public,182.77975291495795,-45.95102399347251,3,6972038119.24151,2000000000.0,A001:DA44 A002:DA51 A003:DV12 A005:DA58 A007:DV05 A008:DV07 A010:DV13 A011:DV21 A015:DA64 A016:DV11 A017:DV14 A019:DA61 A022:DV23 A023:DA42 A024:DA49 A025:DA46 A027:DV06 A033:DA53 A034:DA50 A035:DA43 A036:DA45 A039:DA56 A040:DA55 A041:DA57 A042:DV08 A043:DA60 A044:DV03 A045:DV01 A047:DV15 A048:DV02 A049:DA47 A050:DA62 A058:DA41 A060:DV17 A062:DV10 A066:DV04 A068:DA65 A069:DV16 A070:DA48 A072:DV20 A073:DA54 A074:DV09 A075:DV18 A076:DA59 A082:DA52 A083:DV22 A085:DA63 A088:DV24 T701:PM02 T702:PM03 T703:PM01 T704:PM04,F,2020-02-27T03:03:44.000,1.6159977228975666,"[94.56..96.55GHz,31250.00kHz,852.9uJy/beam@10km/s,34.1uJy/beam@native, XX YY] U [96.46..98.44GHz,31250.00kHz,828.2uJy/beam@10km/s,33.4uJy/beam@native, XX YY] U [106.71..108.69GHz,31250.00kHz,906.8uJy/beam@10km/s,38.4uJy/beam@native, XX YY] U [108.44..110.31GHz,3904.30kHz,799.7uJy/beam@10km/s,35.3uJy/beam@native, XX YY]",107.6985632433412,10794.670283674952,ALMA,ALMaQUEST. IV. The ALMA-MaNGA QUEnching and STar Formation (ALMaQUEST) Survey The ALMaQUEST Survey - II. What drives central starbursts at z ∼ 0? The ALMaQUEST Survey - V. The non-universality of kpc-scale star formation relations and the factors that driv,"Baker, William M. Ellison, Sara L. Lin, Lihwai Su, Yung-Chau Thorp, Mallory D.","Baker, William M.; Maiolino, Roberto; Belfiore, Francesco; Curti, Mirko; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Thorp, Mallory; Pan, Hsi-An Baker, William M.; Maiolino, Roberto; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Belfiore, Francesco; Pan, Hsi-An; Thorp, Mallory Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Scudder, Jillian M.; Sánchez, Sebastian F.; Bluck, Asa F. L.; Maiolino, Roberto Ellison, Sara L.; Lin, Lih

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2018.1.00541.S,ALMA,JAO,ALMA,uid://A001/X133d/Xe8b.source.588015510364880957_8082-6103.spw.25,cube,2,588015510364880957_8082-6103,49.78217315997774,0.955959020000414,0.015308842469507027,Polygon ICRS 49.782173 0.963613 49.789829 0.955959 49.782173 0.948305 49.774518 0.955959,--,--,480,1,2,2.5974896416081967,58497.899179,58502.039046,3265.92,3265.92,0.0026415258711842944,0.002685812096206742,28585.482029130202,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X133d/Xe8b,applicati,--,2018.1.00541.S,Public,180.65813001084166,-44.73196845826679,3,6553960387.929905,1875000000.0,A001:DA44 A002:DA51 A003:DV12 A004:DV18 A005:DA58 A006:DV25 A007:DV05 A008:DV07 A009:DV22 A010:DV13 A011:DV21 A013:DA48 A015:DA64 A016:DV11 A017:DV14 A018:DA54 A019:DA61 A022:DV23 A023:DA42 A024:DA49 A025:DA46 A026:DV19 A027:DV06 A028:DA63 A030:DA62 A031:DA59 A033:DA53 A034:DA50 A035:DA43 A036:DA45 A038:DV24 A039:DA56 A040:DA55 A041:DA57 A042:DV08 A043:DA60 A044:DV03 A045:DV01 A046:DA52 A047:DV15 A048:DV02 A049:DA47 A058:DA41 A060:DV17 A066:DV04 A068:DA65 A069:DV16 A072:DV20 A074:DV09,F,2020-02-05T20:54:33.000,2.5974896416081967,"[97.82..99.81GHz,31250.00kHz,586.3uJy/beam@10km/s,23.8uJy/beam@native, XX YY] U [99.58..101.56GHz,31250.00kHz,579.5uJy/beam@10km/s,23.7uJy/beam@native, XX YY] U [109.83..111.82GHz,31250.00kHz,697.4uJy/beam@10km/s,30uJy/beam@native, XX YY] U [111.62..113.49GHz,3904.30kHz,629.8uJy/beam@10km/s,28.2uJy/beam@native, XX YY]",112.55646958223316,10487.577494565065,ALMA,ALMaQUEST. IV. The ALMA-MaNGA QUEnching and STar Formation (ALMaQUEST) Survey The ALMaQUEST Survey - II. What drives central starbursts at z ∼ 0? The ALMaQUEST Survey - V. The non-universality of kpc-scale star formation relations and the factors that driv,"Baker, William M. Ellison, Sara L. Lin, Lihwai Su, Yung-Chau Thorp, Mallory D.","Baker, William M.; Maiolino, Roberto; Belfiore, Francesco; Curti, Mirko; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Thorp, Mallory; Pan, Hsi-An Baker, William M.; Maiolino, Roberto; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Belfiore, Francesco; Pan, Hsi-An; Thorp, Mallory Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Scudder, Jillian M.; Sánchez, Sebastian F.; Bluck, Asa F. L.; Maiolino, Roberto Ellison,

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2019.1.00260.S,ALMA,JAO,ALMA,uid://A001/X1465/X3509.source.WP_8083_12703.spw.19,image,2,WP_8083_12703,49.92933999999962,0.5654778500000693,0.015304784428837441,Polygon ICRS 49.929340 0.573130 49.936993 0.565478 49.929340 0.557825 49.921687 0.565478,--,--,128,1,2,2.5099327029440244,58807.11577,58807.250348,8890.56,8890.56,0.002667641965732653,0.0027155956657034935,3532.478978062512,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1465/X3509,applicati,--,2019.1.00260.S,Public,181.20959059846066,-44.87701518019899,3,1893903966.3346114,2000000000.0,A001:DA48 A002:DA44 A003:DV23 A004:DA56 A005:DV22 A006:DV13 A007:DV21 A008:DA46 A010:DV15 A011:DV07 A015:DV25 A016:DV24 A017:DV14 A019:DA61 A022:DV01 A023:DV12 A024:DA58 A025:DV10 A026:DV18 A027:DV17 A028:DA59 A030:DA52 A031:DA62 A033:DA49 A034:DV08 A036:DA64 A039:DV16 A040:DA65 A041:DV03 A042:DV05 A043:DA42 A044:DA55 A045:DA57 A046:DA51 A048:DA50 A049:DA63 A050:DV19 A058:DA41 A060:DA47 A062:DV02 A066:DA54 A068:DV20 A069:DA45 A072:DA53 A074:DV09,F,2021-03-09T22:13:31.000,2.5099327029440244,"[97.90..99.89GHz,31250.00kHz,423.4uJy/beam@10km/s,17.2uJy/beam@native, XX YY] U [99.85..101.84GHz,31250.00kHz,415.8uJy/beam@10km/s,17.1uJy/beam@native, XX YY] U [110.40..112.38GHz,31250.00kHz,508uJy/beam@10km/s,21.9uJy/beam@native, XX YY] U [111.60..113.47GHz,1128.91kHz,565.4uJy/beam@10km/s,25.3uJy/beam@native, XX YY]",111.38882193416009,3032.9087429939723,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Chen, Jianhang Klitsch, Anne Thorp, Mallory D.","Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Klitsch, Anne; Davis, Timothy A.; Hamanowicz, Aleksandra; van de Voort, Freeke; Péroux, Céline; Zwaan, Martin A. Klitsch, Anne; Péroux, Céline; Zwaan, Martin A.; Smail, Ian; Nelson, Dylan; Popping, Gergö; Chen, Chian-Chou; Diemer, Benedikt; Ivison, R. J.; Allison, James R.; Muller, Sébastien; Swinbank, A. Mark; Hamanowicz, Aleksandra; Biggs, Andrew D.; 

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [2,2]",Frequency resolution [2],sensitivity_10kms [2],Line sensitivity (native) [2],Polarizations [2]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2017.1.01093.S,ALMA,JAO,ALMA,uid://A001/X1288/Xf05.source.8655-12705.spw.19,cube,2,8655-12705,357.6517333333413,-1.1280897222222368,0.014670924717814638,Circle ICRS 357.651733 -1.128090 0.007335,--,--,480,1,2,2.1400562197502015,58303.409328,58303.446741,2842.56,2842.56,0.0026962727461046685,0.0027424183372160506,28001.82410987244,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1288/Xf05,applicati,--,2017.1.01093.S,Public,91.09630410212674,-60.18462150432066,3,6017766153.233163,1875000000.0,A001:DV07 A002:DA46 A003:DV12 A004:DA52 A006:DA61 A007:DV05 A008:DA42 A009:DV18 A010:DA50 A011:DV21 A013:DV24 A015:DA64 A016:DA44 A017:DA56 A018:DV13 A019:DA58 A022:DV23 A023:DA41 A024:DA49 A025:DV02 A026:DA57 A027:DV06 A028:DA63 A031:DA48 A033:DA53 A034:DA55 A036:DA45 A038:DV22 A039:DV14 A040:DA51 A041:DV19 A042:DV08 A044:DV11 A045:DV01 A046:DA54 A047:DV15 A048:DA62 A049:DV03 A058:DA43 A060:DV17 A062:DV10 A066:DV04 A068:DA65 A069:DV16 A072:DV20 A074:DV09 T701:PM02 T702:PM03 T703:PM01 T704:PM04,F,2019-07-17T15:48:34.000,2.1400562197502015,"[109.26..111.24GHz,31250.00kHz,381uJy/beam@10km/s,16.3uJy/beam@native, XX YY] U [109.32..111.19GHz,3904.30kHz,381uJy/beam@10km/s,16.9uJy/beam@native, XX YY]",110.25227834910038,10706.176027093317,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Baker, William M. Chen, Jianhang Ellison, Sara L. Klitsch, Anne Lin, Lihwai Roberts, Ian D. Su, Yung-Chau Thorp, Mallory D.","Baker, William M.; Maiolino, Roberto; Belfiore, Francesco; Curti, Mirko; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Thorp, Mallory; Pan, Hsi-An Baker, William M.; Maiolino, Roberto; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Belfiore, Francesco; Pan, Hsi-An; Thorp, Mallory Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Scudder, Jillian M.; Sánchez, Sebastian F.; Bluck, Asa F. L.; Maiolino, Roberto Ellison, Sara L.;

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [2,2]",Frequency resolution [2],sensitivity_10kms [2],Line sensitivity (native) [2],Polarizations [2]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2017.1.01093.S,ALMA,JAO,ALMA,uid://A001/X1288/Xefd.source.8616-6104.spw.19,cube,2,8616-6104,322.98052958330265,0.2137666666667376,0.014793224745412156,Circle ICRS 322.980530 0.213767 0.007397,--,--,480,1,2,2.5341352170905935,58293.285862,58293.322618,2812.32,2812.32,0.002718559137599197,0.002765476658817364,27768.757038027266,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1288/Xefd,applicati,--,2017.1.01093.S,Public,54.16823355622182,-34.87632496553882,3,6118424852.236145,1875000000.0,A001:DV07 A002:DA46 A003:DV12 A004:DA52 A005:DV25 A006:DA61 A008:DA42 A010:DA50 A011:DV21 A013:DV24 A015:DA64 A016:DA44 A017:DA56 A018:DV13 A019:DA58 A022:DV23 A023:DA41 A024:DA49 A025:DV02 A027:DV06 A028:DA63 A030:DA59 A031:DA48 A033:DA53 A034:DA55 A036:DA45 A038:DV22 A039:DV14 A040:DA51 A041:DV19 A042:DV08 A043:DA60 A044:DV11 A045:DV01 A046:DA54 A047:DV15 A048:DA62 A049:DV03 A058:DA43 A060:DV17 A062:DV10 A066:DV04 A068:DA65 A069:DV16 A072:DV20,F,2019-06-27T23:41:48.000,2.5341352170905935,"[108.35..110.33GHz,31250.00kHz,422uJy/beam@10km/s,18uJy/beam@native, XX YY] U [108.41..110.28GHz,3904.30kHz,422uJy/beam@10km/s,18.6uJy/beam@native, XX YY]",109.34078968334461,10796.034463820484,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Baker, William M. Chen, Jianhang Ellison, Sara L. Klitsch, Anne Lin, Lihwai Roberts, Ian D. Su, Yung-Chau Thorp, Mallory D.","Baker, William M.; Maiolino, Roberto; Belfiore, Francesco; Curti, Mirko; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Thorp, Mallory; Pan, Hsi-An Baker, William M.; Maiolino, Roberto; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Belfiore, Francesco; Pan, Hsi-An; Thorp, Mallory Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Scudder, Jillian M.; Sánchez, Sebastian F.; Bluck, Asa F. L.; Maiolino, Roberto Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Sánchez, S

...

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2019.1.00260.S,ALMA,JAO,ALMA,uid://A001/X1465/X34e5.source.PM1_8615_1901.spw.19,image,2,PM1_8615_1901,321.0721999999793,1.0283599000000918,0.015222026976726815,Circle ICRS 321.072200 1.028360 0.007611,--,--,128,1,2,2.3128242369498704,58802.900461,58802.920136,1542.24,1542.24,0.002655606833476438,0.0027031248718653706,3548.779361334677,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1465/X34e5,applicati,--,2019.1.00260.S,Public,53.70471150932326,-32.837534694089946,3,1873475769.9849598,2000000000.0,A001:DA48 A002:DA44 A003:DV23 A004:DA56 A005:DV22 A006:DV13 A007:DV21 A008:DA46 A010:DV15 A011:DV07 A015:DV25 A016:DV24 A017:DV14 A019:DA61 A022:DV01 A024:DA58 A027:DV17 A033:DA49 A034:DV08 A035:DA43 A036:DA64 A039:DV16 A040:DA65 A041:DV03 A042:DV05 A043:DA42 A044:DA55 A045:DA57 A046:DA51 A047:DV06 A048:DA50 A049:DA63 A050:DV19 A058:DA41 A060:DA47 A062:DV02 A066:DA54 A068:DV20 A069:DA45 A070:DA59 A072:DA53 A088:DV11,F,2021-03-06T12:58:33.000,2.3128242369498704,"[98.54..100.53GHz,31250.00kHz,670.1uJy/beam@10km/s,27.3uJy/beam@native, XX YY] U [100.30..102.29GHz,31250.00kHz,654.3uJy/beam@10km/s,26.9uJy/beam@native, XX YY] U [110.91..112.89GHz,31250.00kHz,820.1uJy/beam@10km/s,35.4uJy/beam@native, XX YY] U [112.10..113.98GHz,1128.91kHz,630.8uJy/beam@10km/s,28.3uJy/beam@native, XX YY]",111.89813160124298,3019.1821917154107,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Chen, Jianhang Klitsch, Anne Thorp, Mallory D.","Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Klitsch, Anne; Davis, Timothy A.; Hamanowicz, Aleksandra; van de Voort, Freeke; Péroux, Céline; Zwaan, Martin A. Klitsch, Anne; Péroux, Céline; Zwaan, Martin A.; Smail, Ian; Nelson, Dylan; Popping, Gergö; Chen, Chian-Chou; Diemer, Benedikt; Ivison, R. J.; Allison, James R.; Muller, Sébastien; Swinbank, A. Mark; Hamanowicz, Aleksandra; Biggs, Andrew D.; Dutta, Rajeshwari Thorp, Mallory D.; Ellison, Sara L.; Pan, Hsi-An; Lin, 

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [2,2]",Frequency resolution [2],sensitivity_10kms [2],Line sensitivity (native) [2],Polarizations [2]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2017.1.01093.S,ALMA,JAO,ALMA,uid://A001/X1288/Xf09.source.8728-3701.spw.19,cube,2,8728-3701,57.69902791668954,-7.028786666667182,0.014429174182394236,Circle ICRS 57.699028 -7.028787 0.007215,--,--,480,1,2,2.128994489859118,58272.681113,58272.693072,907.2,907.2,0.002652208032966512,0.0026968506758088596,28471.50366666822,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1288/Xf09,applicati,--,2017.1.01093.S,Public,196.07898230406806,-43.01042842661591,3,5821782888.577181,1875000000.0,A001:DV07 A002:DA46 A003:DV12 A004:DA52 A006:DA61 A007:DV05 A010:DA50 A011:DV21 A013:DV24 A015:DA64 A016:DA44 A017:DA56 A018:DV13 A019:DA58 A022:DV23 A024:DA49 A027:DV06 A028:DA63 A030:DA59 A031:DA48 A033:DA53 A034:DA55 A036:DA45 A038:DV22 A039:DV14 A040:DA51 A041:DV19 A043:DA60 A044:DV11 A045:DV01 A046:DA54 A047:DV15 A048:DA62 A049:DV03 A058:DA43 A060:DV17 A062:DV10 A066:DV04 A068:DA65 A072:DV20 A074:DV09 T702:PM03 T703:PM01 T704:PM04,F,2019-06-13T19:33:31.000,2.128994489859118,"[111.11..113.09GHz,31250.00kHz,1.1mJy/beam@10km/s,46.1uJy/beam@native, XX YY] U [111.16..113.04GHz,3904.30kHz,1.1mJy/beam@10km/s,47.6uJy/beam@native, XX YY]",112.09947673921536,10529.56182117522,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Baker, William M. Chen, Jianhang Ellison, Sara L. Klitsch, Anne Lin, Lihwai Roberts, Ian D. Su, Yung-Chau Thorp, Mallory D.","Baker, William M.; Maiolino, Roberto; Belfiore, Francesco; Curti, Mirko; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Thorp, Mallory; Pan, Hsi-An Baker, William M.; Maiolino, Roberto; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Belfiore, Francesco; Pan, Hsi-An; Thorp, Mallory Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Scudder, Jillian M.; Sánchez, Sebastian F.; Bluck, Asa F. L.; Maiolino, Roberto Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Sánchez, Sebastian F.; Bl

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2019.1.00260.S,ALMA,JAO,ALMA,uid://A001/X1465/X34f9.source.PM2_7975_6104.spw.19,image,2,PM2_7975_6104,324.89156999998664,10.483480000000288,0.016119136137783038,Circle ICRS 324.891570 10.483480 0.008060,--,--,128,1,2,1.7513278633224256,58790.954214,58790.96807,1118.88,1118.88,0.0028078743973483852,0.002861051786770979,3352.9292462007643,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1465/X34f9,applicati,--,2019.1.00260.S,Public,65.25548642838066,-30.19696129125304,3,2100785117.0677538,2000000000.0,A001:DA48 A002:DA44 A005:DV22 A006:DV13 A007:DV21 A008:DA46 A010:DV15 A011:DV07 A015:DV25 A016:DV24 A019:DA61 A022:DV01 A023:DV12 A024:DA58 A025:DV10 A026:DV18 A027:DV17 A033:DA49 A034:DV08 A035:DA43 A036:DA64 A039:DV16 A040:DA65 A041:DV03 A042:DV05 A043:DA42 A044:DA55 A045:DA57 A047:DV06 A048:DA50 A049:DA63 A050:DV19 A058:DA41 A060:DA47 A062:DV02 A066:DA54 A069:DA45 A070:DA59 A072:DA53 A073:DV23 A074:DV09 A076:DA62 A082:DA60 A083:DA51 A085:DA52 A088:DV11,F,2021-02-28T23:28:07.000,1.7513278633224256,"[92.91..94.89GHz,31250.00kHz,719.8uJy/beam@10km/s,28.5uJy/beam@native, XX YY] U [94.74..96.73GHz,31250.00kHz,705.3uJy/beam@10km/s,28.2uJy/beam@native, XX YY] U [104.78..106.77GHz,31250.00kHz,738.2uJy/beam@10km/s,31uJy/beam@native, XX YY] U [105.91..107.78GHz,1128.91kHz,742.3uJy/beam@10km/s,32.4uJy/beam@native, XX YY]",105.77623715058351,3195.706646215608,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Chen, Jianhang Klitsch, Anne Thorp, Mallory D.","Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Klitsch, Anne; Davis, Timothy A.; Hamanowicz, Aleksandra; van de Voort, Freeke; Péroux, Céline; Zwaan, Martin A. Klitsch, Anne; Péroux, Céline; Zwaan, Martin A.; Smail, Ian; Nelson, Dylan; Popping, Gergö; Chen, Chian-Chou; Diemer, Benedikt; Ivison, R. J.; Allison, James R.; Muller, Sébastien; Swinbank, A. Mark; Hamanowicz, Aleksandra; Biggs, Andrew D.; Dutta, Rajeshwari Thorp, Mallory D.; E

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2019.1.00260.S,ALMA,JAO,ALMA,uid://A001/X1465/X34f5.source.PM2_8153_12702.spw.19,image,2,PM2_8153_12702,40.30401999994972,-0.87703000000046,0.015511159388851097,Circle ICRS 40.304020 -0.877030 0.007756,--,--,128,1,2,2.5713308174393448,58804.063888,58804.080005,1239.84,1239.84,0.0027032643975281353,0.0027525193835548855,3485.074366883621,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X1465/X34f5,applicati,--,2019.1.00260.S,Public,172.61894753359536,-52.81736468311583,3,1945334543.8593028,2000000000.0,A001:DA48 A002:DA44 A004:DA56 A005:DV22 A006:DV13 A007:DV21 A008:DA46 A010:DV15 A011:DV07 A015:DV25 A016:DV24 A017:DV14 A019:DA61 A022:DV01 A023:DV12 A024:DA58 A025:DV10 A027:DV17 A031:DA62 A033:DA49 A034:DV08 A035:DA43 A036:DA64 A039:DV16 A040:DA65 A041:DV03 A042:DV05 A043:DA42 A044:DA55 A045:DA57 A046:DA51 A047:DV06 A048:DA50 A049:DA63 A050:DV19 A058:DA41 A060:DA47 A062:DV02 A066:DA54 A068:DV20 A069:DA45 A072:DA53 A074:DV09,F,2021-03-05T23:56:33.000,2.5713308174393448,"[96.59..98.57GHz,31250.00kHz,707.8uJy/beam@10km/s,28.6uJy/beam@native, XX YY] U [98.52..100.50GHz,31250.00kHz,684.4uJy/beam@10km/s,27.9uJy/beam@native, XX YY] U [108.92..110.90GHz,31250.00kHz,804.6uJy/beam@10km/s,34.4uJy/beam@native, XX YY] U [110.10..111.97GHz,1128.91kHz,800.7uJy/beam@10km/s,35.6uJy/beam@native, XX YY]",109.90790937116552,3074.224935052848,ALMA,ALMACAL - VI. Molecular gas mass density across cosmic time via a blind search for intervening molecular absorbers ALMACAL - X. Constraints on molecular gas in the low-redshift circumgalactic medium ALMACAL IX: Multiband ALMA survey for dusty star-forming,"Chen, Jianhang Klitsch, Anne Thorp, Mallory D.","Chen, Jianhang; Ivison, R. J.; Zwaan, Martin A.; Smail, Ian; Klitsch, Anne; Péroux, Céline; Popping, Gergö; Biggs, Andrew D.; Szakacs, Roland; Hamanowicz, Aleksandra; Lagos, Claudia Klitsch, Anne; Davis, Timothy A.; Hamanowicz, Aleksandra; van de Voort, Freeke; Péroux, Céline; Zwaan, Martin A. Klitsch, Anne; Péroux, Céline; Zwaan, Martin A.; Smail, Ian; Nelson, Dylan; Popping, Gergö; Chen, Chian-Chou; Diemer, Benedikt; Ivison, R. J.; Allison, James R.; Muller, Sébastien; Swinbank, A. Mark; Hamanowicz, Aleksandra; Biggs, Andrew D.; Dutta, Rajeshwari Thorp, Mallory D.; Ellison, Sara L.; Pan, Hsi-

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [2,2]",Frequency resolution [2],sensitivity_10kms [2],Line sensitivity (native) [2],Polarizations [2]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2018.1.00558.S,ALMA,JAO,ALMA,uid://A001/X133d/Xf51.source.8077-9101.spw.19,cube,2,8077-9101,41.64311208333703,-0.8435372222222467,0.014755272379185023,Polygon ICRS 41.643112 -0.850915 41.635734 -0.843537 41.643112 -0.836160 41.650491 -0.843537,--,--,480,1,2,2.449759976812857,58502.963186,58503.017616,3507.84,3507.84,0.0026903185769601872,0.0027362696779855886,28058.275005098763,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X133d/Xf51,applicati,--,2018.1.00558.S,Public,174.17513833794175,-51.87382621362544,3,6088564893.865721,1875000000.0,A001:DA44 A002:DA51 A004:DV18 A005:DA58 A006:DV25 A007:DV05 A008:DV07 A009:DV22 A010:DV13 A011:DV21 A013:DA48 A015:DA64 A016:DV11 A017:DV14 A018:DA54 A019:DA61 A022:DV23 A023:DA42 A024:DA49 A025:DA46 A026:DV19 A027:DV06 A028:DA63 A030:DA62 A031:DA59 A033:DA53 A034:DA50 A035:DA43 A036:DA45 A038:DV24 A039:DA56 A040:DA55 A041:DA57 A042:DV08 A043:DA60 A044:DV03 A046:DA52 A047:DV15 A048:DV02 A049:DA47 A058:DA41 A060:DV17 A062:DV10 A066:DV04 A068:DA65 A069:DV16 A072:DV20 A074:DV09,F,2020-02-20T07:58:24.000,2.449759976812857,"[107.81..109.79GHz,31250.00kHz,643.2uJy/beam@10km/s,27.4uJy/beam@native, XX YY] U [109.56..111.43GHz,3904.30kHz,696uJy/beam@10km/s,30.9uJy/beam@native, XX YY]",110.49814045693535,10684.636099172938,ALMA,ALMaQUEST. IV. The ALMA-MaNGA QUEnching and STar Formation (ALMaQUEST) Survey The ALMaQUEST Survey - II. What drives central starbursts at z ∼ 0? The ALMaQUEST Survey - V. The non-universality of kpc-scale star formation relations and the factors that driv,"Baker, William M. Ellison, Sara L. Lin, Lihwai Su, Yung-Chau Thorp, Mallory D.","Baker, William M.; Maiolino, Roberto; Belfiore, Francesco; Curti, Mirko; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Thorp, Mallory; Pan, Hsi-An Baker, William M.; Maiolino, Roberto; Bluck, Asa F. L.; Lin, Lihwai; Ellison, Sara L.; Belfiore, Francesco; Pan, Hsi-An; Thorp, Mallory Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Scudder, Jillian M.; Sánchez, Sebastian F.; Bluck, Asa F. L.; Maiolino, Roberto Ellison, Sara L.; Lin, Lihwai; Thorp, Mallory D.; Pan, Hsi-An; Sánchez, Sebastian F.; Bluck, Asa F. L.; Belfiore, Francesco Ellison, Sara L.; Thorp, Mallory D.; Lin, Lihwai; Pan, Hsi-An; Bluck, Asa F. L.; Scudder

In [57]:
n

50

### 3. Search using `runTargetQueriesWithLines`

In [11]:
# same thing as before?
sdssLineQuery = archiveSearch(targets=targetCoords)

In [12]:
sdssLineQuery.runQueriesWithLines(restFreqs=rest_frequencies, radius = 30*u.arcsec, line_names=line_names, science=True)

ALMA archive querying: 100%|██████████| 51/51 [01:28<00:00,  1.74s/ target]

coord=(49.82143689 deg -0.96964645 deg) radius=0.008333333333333333deg : No result
coord=(49.78215908 deg 0.95595472 deg) radius=0.008333333333333333deg : No result
coord=(49.92932926 deg 0.56545578 deg) radius=0.008333333333333333deg : No result
coord=(357.65176944 deg -1.12805687 deg) radius=0.008333333333333333deg : No result
coord=(322.98051569 deg 0.21380748 deg) radius=0.008333333333333333deg : No result
coord=(323.40752218 deg 1.07095399 deg) radius=0.008333333333333333deg : No result
coord=(126.46116245 deg 18.16669719 deg) radius=0.008333333333333333deg : No result
coord=(47.0294424 deg 0.45621817 deg) radius=0.008333333333333333deg : No result
coord=(319.91975399 deg 0.1209354 deg) radius=0.008333333333333333deg : No result
coord=(322.90884307 deg -0.60820117 deg) radius=0.008333333333333333deg : No result
coord=(49.94013207 deg -0.07720499 deg) radius=0.008333333333333333deg : No result
coord=(50.63658867 deg -0.00119968 deg) radius=0.008333333333333333deg : No result
coord=

TypeError: replace() argument 1 must be str, not numpy.bytes_

**Display the results**

In [13]:
n=0 #counter
for targetname in sdssLineQuery.targets.keys():
    if len(sdssLineQuery.queryResults[targetname])>=1:
        n = n+1
        display(sdssLineQuery.queryResults[targetname])

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [1,2]",Frequency resolution [1],sensitivity_10kms [1],Line sensitivity (native) [1],Polarizations [1]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2022.1.01515.S,ALMA,JAO,ALMA,uid://A001/X35f5/X9f7.source.BGALX312p00MPAID330602.spw.13,cube,2,BGALX312p00MPAID330602,311.78094583338293,0.3004669444445401,0.026366907071111206,Circle ICRS 311.780946 0.300467 0.013183,--,--,2048,1,2,11.87205825269995,60164.259782,60164.268454,272.16,272.16,0.0028372478636296593,0.0028643417601128367,185432.4852982429,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X35f5/X9f7,applicati,--,2022.1.01515.S,Proprietary,47.3596652468795,-25.341342070931642,3,956272504.6578152,1000000000.0,J501:CM10 J502:CM02 J503:CM03 J504:CM12 J505:CM08 J506:CM05 N602:CM01 N603:CM09 N604:CM11 N605:CM04 N606:CM06,F,2024-08-17T14:25:21.000,11.87205825269995,"[104.66..105.66GHz,564.45kHz,14.9mJy/beam@10km/s,883.2uJy/beam@native, XX YY]",105.16338343270232,1616.7202716278364,ALMA,,,,,--,"The rapid decline in baryon content towards the upper end of the galaxy mass function is one of the major open questions of galaxy evolution. It has been attributed to phenomena as diverse as environment, stellar mass-surface densities, bulge potential, and AGN feedback. We will use the ACA for a CO(1-0) survey of a purely mass-selected sample of 197 galaxies at z<0.3 with M_stellar 10^11-12 Msun, where the baryon deficit in galaxies is strongest. Our survey extends the xCOLD-GASS survey, which has been very successful in characterizing the cold gas in galaxies with stellar masses 10^9-10 Msun, but includes few galaxies above >10^11 Msun. Our sources fall within the SDSS Stripe 82, and have all SDSS spectroscopy, LOFAR and VLA radio imaging, and a rich sets of other photometric constraints. We will measure the local H2 mass function out to the highest masses, and obtain, e.g., integrated molecular gas masses, mass fractions, and gas consumption times as a function of the stellar, radio, and AGN properties of our sources, placing unbiased constraints on each of these characterstics in regulating the gas content and star formation efficiency in the most massive galaxies today.",RGALX317_c_03_7M,"Nesvadba, Nicole; Zovaro, Henry;",0.883231461380426,5.186,uid://A001/X35f5/X9f6,uid://A001/X35f5/X9f7,uid://A002/X10b22d7/X14fd,An unbiased census of the molecular gas content in the most massive galaxies in the nearby Universe,S,TARG

obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,lastModified,pi_name,collections,pi_userid,obs_start_date,"Frequency ranges [4,2]",Frequency resolution [4],sensitivity_10kms [4],Line sensitivity (native) [4],Polarizations [4]
,,,,,,,,deg,deg,deg,,,,,,,arcsec,d,d,s,s,m,m,,,,,kbyte,,,deg,deg,,m,Hz,,,,arcsec,GHz,GHz,m / s,,,,,,,,,,mJy / beam,mm,,,,,,,,arcsec,,,,,,,,,,GHz,kHz,mJy / beam,mJy / beam,
str33,str4,str3,str4,str64,str5,int32,str256,float64,float64,float64,object,int32,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,str35,str72,str9,int32,str64,str11,float64,float64,str30,float64,float64,str660,str1,object,float64,str4000,float64,float64,str256,str256,str256,str4000,str4000,int32,str4000,str128,str2000,float64,float32,str64,str64,str32,str256,str16,str256,str1,float64,str1,str4000,str200,str200,str32,str256,str128,str256,datetime64[D],float64,float64,float64,float64,str5
ADS/JAO.ALMA#2022.1.00510.S,ALMA,JAO,ALMA,uid://A001/X2d20/X2665.source.Coma841.spw.18,cube,2,Coma841,194.73587500001832,27.821972222223526,0.01205946964766606,Polygon ICRS 194.736729 27.815990 194.729111 27.821216 194.735020 27.827954 194.742639 27.822728,--,--,2048,1,2,6.396036918776277,60146.930567,60167.8891,241.92,241.92,0.0013329505994211417,0.0013449052116441726,199221.4443968865,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid://A001/X2d20/X2665,applicati,--,2022.1.00510.S,Proprietary,55.98342085331166,88.19653443172047,6,400109953.8590517,2000000000.0,J501:CM10 J502:CM02 J503:CM03 J504:CM12 J505:CM08 J506:CM05 N602:CM01 N603:CM09 N604:CM11 N605:CM04 N606:CM06,F,2024-08-31T14:27:23.000,6.396036918776277,"[220.43..222.43GHz,1128.91kHz,30.6mJy/beam@10km/s,1.9mJy/beam@native, XX YY] U [222.91..224.91GHz,1128.91kHz,28.2mJy/beam@10km/s,1.7mJy/beam@native, XX YY] U [235.49..237.49GHz,1128.91kHz,29.8mJy/beam@10km/s,1.9mJy/beam@native, XX YY] U [237.43..239.43GHz,1128.91kHz,30.8mJy/beam@10km/s,1.9mJy/beam@native, XX YY]",223.9093169967461,1425.4576429511405,ALMA,,,,,--,"We propose to observe CO(2-1) in 142 galaxies in the Coma cluster, selected to have ongoing star formation according to 12 micron observations. The sample is representative, goes out to ~R200 (the radius at which the density is equal to the critical density of the local Universe, multiplied by 200) and spans a wide range of both stellar masses and star formation rates. The Coma cluster is by far the most massive cluster in the local universe. We aim to study the effects of such an extreme environment on the molecular gas, the direct fuel for star formation. Moreover, we aim to compare the molecular gas in Coma galaxies to that in Fornax and Virgo galaxies, and to identify the quenching mechanisms that dominate in different mass galaxy clusters. Ample ancillary data is available, such as Ha, 12 micron, FIR, and HI, allowing us to study scaling relations and ISM phase ratios, and compare these to other clusters as well as the field. Additionally, the Coma cluster is known to harbour several cases of extreme ram pressure stripping, allowing us to study the state of the molecular gas in such objects.",Coma4_a_06_7M,"Healy, Julia; Mok, Angus; Wilson, Christine; Molnar, Danie

In [14]:
print("galaxies with 12CO J=1-0:", n)

galaxies with 12CO J=1-0: 2


In [41]:
sdssLineQuery.queryResults['coord=(20.96776532 deg 13.29376443 deg) radius=0.005555555555555556deg']

KeyError: 'coord=(20.96776532 deg 13.29376443 deg) radius=0.005555555555555556deg'

In [ ]:
k=0 #counter
for targetname in herusLineQuery.targets.keys():
    if len(herusLineQuery.queryResults[targetname])>=1:
        # select only galaxies with all three lines
        if ((True in herusLineQuery.queryResults[targetname]['12CO J=1-0']) & 
            (True in herusLineQuery.queryResults[targetname]['13CO J=1-0']) &
            (True in herusLineQuery.queryResults[targetname]['C18O J=1-0'])):
            k = k+1
            display(herusLineQuery.queryResults[targetname])


In [ ]:
print("galaxies with 12CO, 13CO, AND C18O:", k)

## Test the time

In [26]:
import pymysql
import numpy as np

#Query MySQL for mock survey statmorph results
db = pymysql.connect(host = 'lauca.phys.uvic.ca', db = 'sdss', user = 'swilkinson', passwd = '123Sdss!@#')
#x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47'
#x = 'SELECT DISTINCT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u, dr15_ALMA_spaxels a WHERE u.objID = a.objID AND u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47'
#x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47 AND u.ra > 20 and u.ra < 35'
x = 'SELECT u.objID, u.ra, u.decl, u.total_mass_med, u.z_spec FROM  dr7_uberuber u WHERE u.total_mass_med > 1 AND u.z_spec < 0.5 AND u.decl < 47 LIMIT 5000'
c = db.cursor()
c.execute(x)
db_data = c.fetchall()
c.close()
db.close()
	
# transpose queried table
data = np.array(db_data, dtype = float).T
table_id = np.array(db_data, dtype = str).T[0]

In [27]:
ra_sdss = data[1]
dec_sdss = data[2]

In [28]:
import sys
#sys.path = ['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/tools/ArchiveDev'] + sys.path
sys.path = ['/arc/home/swilkinson/SDSS-ALMA/ALMAxmatch/'] + sys.path
from ALMAxmatch import archiveSearch


In [29]:
from astropy import units as u
from astropy.coordinates import SkyCoord

# archive search impact parameter
searchradius = '30arcsec'

# SkyCoord objects
#targetCoords = [[SkyCoord(ra=ra*u.degree, dec=dec*u.degree), searchradius]
#                   for ra, dec in zip(ra_sdss, dec_sdss)]

targetCoords = [[SkyCoord(ra=ra*u.degree, dec=dec*u.degree), 30*u.arcsec] for ra, dec in zip(ra_sdss, dec_sdss)]

In [30]:
# search using SkyCoord array
sdssALMAQuery = archiveSearch(targets=targetCoords)

In [31]:
import time

start = time.time()
sdssALMAQuery.runQueries(science=True, public=True, radius = 30*u.arcsec)
end = time.time()
print(f'Querying {len(mycat)} galaxies took {(start-end)/60} min.')

ALMA archive querying: 100%|██████████| 5000/5000 [2:20:02<00:00,  1.68s/ target]  


coord=(237.26195851 deg 0.25982931 deg) radius=0.008333333333333333deg : No result
coord=(237.21732351 deg 0.34432801 deg) radius=0.008333333333333333deg : No result
coord=(237.15985346 deg 0.31282368 deg) radius=0.008333333333333333deg : No result
coord=(236.90034647 deg 0.36903487 deg) radius=0.008333333333333333deg : No result
coord=(236.88772799 deg 0.39000637 deg) radius=0.008333333333333333deg : No result
coord=(236.88157401 deg 0.34448976 deg) radius=0.008333333333333333deg : No result
coord=(236.85214508 deg 0.32747237 deg) radius=0.008333333333333333deg : No result
coord=(236.7759235 deg 0.3477304 deg) radius=0.008333333333333333deg : No result
coord=(236.64558889 deg 0.25713752 deg) radius=0.008333333333333333deg : No result
coord=(236.29859769 deg 0.40884912 deg) radius=0.008333333333333333deg : No result
coord=(236.36318813 deg 0.21511341 deg) radius=0.008333333333333333deg : No result
coord=(236.3491714 deg 0.29436021 deg) radius=0.008333333333333333deg : No result
coord=(

NameError: name 'mycat' is not defined

In [ ]:
#

In [ ]:
#